## Proyecto 1 - Security Data Science

In [1]:
#Librerias que se van a utilizar. 
import pandas as pd
import numpy as np
import  re
import neattext as nt
import unicodedata
import nltk
from sklearn import metrics, model_selection, tree
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#Carga de datos
df1 = pd.read_csv('datasets/dataset-part1.csv')
df2 = pd.read_csv('datasets/dataset-part2.csv')
df3 = pd.read_csv('datasets/dataset-part3.csv')
df4 = pd.read_csv('datasets/dataset-part4.csv')

In [12]:
#Analisis Exploratorio
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3570666 entries, 0 to 3570665
Data columns (total 50 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   BIFLOW_DIRECTION            int64 
 1   DIRECTION                   int64 
 2   DST_TO_SRC_SECOND_BYTES     object
 3   FIREWALL_EVENT              int64 
 4   FIRST_SWITCHED              int64 
 5   FLOW_ACTIVE_TIMEOUT         int64 
 6   FLOW_DURATION_MICROSECONDS  int64 
 7   FLOW_DURATION_MILLISECONDS  int64 
 8   FLOW_END_MILLISECONDS       int64 
 9   FLOW_END_SEC                int64 
 10  FLOW_ID                     int64 
 11  FLOW_INACTIVE_TIMEOUT       int64 
 12  FLOW_START_MILLISECONDS     int64 
 13  FLOW_START_SEC              int64 
 14  FRAME_LENGTH                int64 
 15  IN_BYTES                    int64 
 16  IN_PKTS                     int64 
 17  IPV4_DST_ADDR               object
 18  IPV4_SRC_ADDR               object
 19  L4_DST_PORT                 int64 
 20  L4

In [13]:
df1.head()

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,40,0,1616660040,120,339,0,1616660040010,1616660040,...,1460,0,0,0,0,0,Unknown,1,2293398,Normal flow
1,1,0,",",0,1616660040,120,0,0,1616660040068,1616660040,...,0,0,0,0,40,0,ICMP,1,2293400,Normal flow
2,1,0,104,0,1616660040,120,44725,44,1616660040114,1616660040,...,0,0,0,0,0,0,TLS,1,2293404,Normal flow
3,1,0,",",0,1616660040,120,0,0,1616660040122,1616660040,...,1440,0,8,0,40,0,Unknown,1,2293407,Normal flow
4,1,0,40,0,1616660040,120,1114,1,1616660040184,1616660040,...,0,0,0,0,0,0,TLS,1,2293409,Normal flow


In [14]:
df1

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,40,0,1616660040,120,339,0,1616660040010,1616660040,...,1460,0,0,0,0,0,Unknown,1,2293398,Normal flow
1,1,0,",",0,1616660040,120,0,0,1616660040068,1616660040,...,0,0,0,0,40,0,ICMP,1,2293400,Normal flow
2,1,0,104,0,1616660040,120,44725,44,1616660040114,1616660040,...,0,0,0,0,0,0,TLS,1,2293404,Normal flow
3,1,0,",",0,1616660040,120,0,0,1616660040122,1616660040,...,1440,0,8,0,40,0,Unknown,1,2293407,Normal flow
4,1,0,40,0,1616660040,120,1114,1,1616660040184,1616660040,...,0,0,0,0,0,0,TLS,1,2293409,Normal flow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3570661,1,0,"200,,,,,,,,,,,,,,,,,,,,200,,,,,,,,,,,,,,,,,,,,...",0,1616674440,120,101430708,101430,1616674541712,1616674541,...,0,0,0,0,0,0,STUN.SkypeCall,1,5870422,Normal flow
3570662,1,0,"200,,,,,,,,,,,,,,,,,,,,200,,,,,,,,,,,,,,,,,,,,...",0,1616674440,120,101332421,101332,1616674541823,1616674541,...,0,0,0,0,0,0,STUN.SkypeCall,1,5870423,Normal flow
3570663,1,0,"108,,,,,,,,,,,108,,,,,,,,,,110,108,,,,,,,,,,,1...",0,1616674440,120,108700564,108700,1616674549429,1616674549,...,0,0,0,0,0,0,TLS.Facebook,1,5870424,Normal flow
3570664,1,0,"200,,,,,,,,,,,,,,,,,,,,,200,,,,,,,,,,,,,,,,,,,...",0,1616674440,120,101391591,101391,1616674542194,1616674542,...,0,0,0,0,0,0,STUN.SkypeCall,1,5870425,Normal flow


In [17]:
df2.info()
df2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8637207 entries, 0 to 8637206
Data columns (total 50 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   BIFLOW_DIRECTION            int64 
 1   DIRECTION                   int64 
 2   DST_TO_SRC_SECOND_BYTES     object
 3   FIREWALL_EVENT              int64 
 4   FIRST_SWITCHED              int64 
 5   FLOW_ACTIVE_TIMEOUT         int64 
 6   FLOW_DURATION_MICROSECONDS  int64 
 7   FLOW_DURATION_MILLISECONDS  int64 
 8   FLOW_END_MILLISECONDS       int64 
 9   FLOW_END_SEC                int64 
 10  FLOW_ID                     int64 
 11  FLOW_INACTIVE_TIMEOUT       int64 
 12  FLOW_START_MILLISECONDS     int64 
 13  FLOW_START_SEC              int64 
 14  FRAME_LENGTH                int64 
 15  IN_BYTES                    int64 
 16  IN_PKTS                     int64 
 17  IPV4_DST_ADDR               object
 18  IPV4_SRC_ADDR               object
 19  L4_DST_PORT                 int64 
 20  L4

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,",,48,,,,,,,,,,40,,,,,,,,,,40,,,,,,,,,,40,,,,,,...",0,1618174987,120,104116914,104116,1618175092090,1618175092,...,1460,1460,0,0,0,164,TLS,1,77251491,Normal flow
1,1,0,",,,,,,,,,,,",0,1618175067,120,9529030,9529,1618175077010,1618175077,...,0,0,0,0,0,0,STUN.Messenger,1,77251492,Normal flow
2,1,0,",,,,,,,,,,,",0,1618175067,120,9529169,9529,1618175077010,1618175077,...,0,0,0,0,192,0,ICMP,1,77251493,Normal flow
3,1,0,",",0,1618175077,120,0,0,1618175077021,1618175077,...,0,0,0,0,72,0,ICMP,1,77251494,Normal flow
4,1,0,",",0,1618175077,120,0,0,1618175077069,1618175077,...,0,0,0,0,0,0,Unknown,1,77251495,Normal flow


In [18]:
df2

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,",,48,,,,,,,,,,40,,,,,,,,,,40,,,,,,,,,,40,,,,,,...",0,1618174987,120,104116914,104116,1618175092090,1618175092,...,1460,1460,0,0,0,164,TLS,1,77251491,Normal flow
1,1,0,",,,,,,,,,,,",0,1618175067,120,9529030,9529,1618175077010,1618175077,...,0,0,0,0,0,0,STUN.Messenger,1,77251492,Normal flow
2,1,0,",,,,,,,,,,,",0,1618175067,120,9529169,9529,1618175077010,1618175077,...,0,0,0,0,192,0,ICMP,1,77251493,Normal flow
3,1,0,",",0,1618175077,120,0,0,1618175077021,1618175077,...,0,0,0,0,72,0,ICMP,1,77251494,Normal flow
4,1,0,",",0,1618175077,120,0,0,1618175077069,1618175077,...,0,0,0,0,0,0,Unknown,1,77251495,Normal flow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8637202,1,0,"240,,,,,,,,,,,,,,240",0,1618262463,120,14015728,14015,1618262477935,1618262477,...,0,0,0,0,0,0,DNS.UbuntuONE,1,85888693,Normal flow
8637203,1,0,",",0,1618262477,120,0,0,1618262477940,1618262477,...,0,0,0,0,40,0,Unknown,1,85888694,Normal flow
8637204,1,0,"1259,592,52,,224",0,1618262473,120,4921754,4921,1618262477954,1618262477,...,1460,1410,7,7,40,0,SSH,1,85888695,Normal flow
8637205,1,0,205,0,1618262477,120,376,0,1618262477977,1618262477,...,0,0,0,0,128,0,DNS,1,85888696,Normal flow


In [19]:
df3.info()
df3.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330692 entries, 0 to 1330691
Data columns (total 50 columns):
 #   Column                      Non-Null Count    Dtype 
---  ------                      --------------    ----- 
 0   BIFLOW_DIRECTION            1330692 non-null  int64 
 1   DIRECTION                   1330692 non-null  int64 
 2   DST_TO_SRC_SECOND_BYTES     1330692 non-null  object
 3   FIREWALL_EVENT              1330692 non-null  int64 
 4   FIRST_SWITCHED              1330692 non-null  int64 
 5   FLOW_ACTIVE_TIMEOUT         1330692 non-null  int64 
 6   FLOW_DURATION_MICROSECONDS  1330692 non-null  int64 
 7   FLOW_DURATION_MILLISECONDS  1330692 non-null  int64 
 8   FLOW_END_MILLISECONDS       1330692 non-null  int64 
 9   FLOW_END_SEC                1330692 non-null  int64 
 10  FLOW_ID                     1330692 non-null  int64 
 11  FLOW_INACTIVE_TIMEOUT       1330692 non-null  int64 
 12  FLOW_START_MILLISECONDS     1330692 non-null  int64 
 13  FLOW_START_S

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,138,0,1622174400,120,682,0,1622174400001,1622174400,...,0,0,0,0,0,0,DNS,1,1365973,Normal flow
1,1,0,75,0,1622174400,120,579,0,1622174400001,1622174400,...,0,0,0,0,0,0,DNS,1,1365974,Normal flow
2,1,0,40,0,1622174400,120,213,0,1622174400053,1622174400,...,0,0,0,0,40,40,Unknown,1,1365976,Normal flow
3,1,0,166,0,1622174400,120,257,0,1622174400109,1622174400,...,0,0,0,0,0,0,DNS,1,1365978,Normal flow
4,1,0,",",0,1622174400,120,0,0,1622174400114,1622174400,...,0,0,0,0,196,0,ICMP,1,1365979,Normal flow


In [21]:
df4.info()
df4.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26725246 entries, 0 to 26725245
Data columns (total 50 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   BIFLOW_DIRECTION            int64 
 1   DIRECTION                   int64 
 2   DST_TO_SRC_SECOND_BYTES     object
 3   FIREWALL_EVENT              int64 
 4   FIRST_SWITCHED              int64 
 5   FLOW_ACTIVE_TIMEOUT         int64 
 6   FLOW_DURATION_MICROSECONDS  int64 
 7   FLOW_DURATION_MILLISECONDS  int64 
 8   FLOW_END_MILLISECONDS       int64 
 9   FLOW_END_SEC                int64 
 10  FLOW_ID                     int64 
 11  FLOW_INACTIVE_TIMEOUT       int64 
 12  FLOW_START_MILLISECONDS     int64 
 13  FLOW_START_SEC              int64 
 14  FRAME_LENGTH                int64 
 15  IN_BYTES                    int64 
 16  IN_PKTS                     int64 
 17  IPV4_DST_ADDR               object
 18  IPV4_SRC_ADDR               object
 19  L4_DST_PORT                 int64 
 20  

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,138,0,1622433600,120,857,0,1622433600002,1622433600,...,0,0,0,0,0,0,DNS,1,7062121,Normal flow
1,1,0,75,0,1622433600,120,924,0,1622433600002,1622433600,...,0,0,0,0,0,0,DNS,1,7062122,Normal flow
2,1,0,",",0,1622433600,120,0,0,1622433600007,1622433600,...,0,0,0,0,40,0,Unknown,1,7062123,Normal flow
3,1,0,166,0,1622433600,120,513,0,1622433600090,1622433600,...,0,0,0,0,0,0,DNS,1,7062127,Normal flow
4,1,0,52,0,1622433600,120,26595,26,1622433600097,1622433600,...,0,0,0,0,0,0,TLS.Google,1,7062128,Normal flow


In [22]:
df4

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,138,0,1622433600,120,857,0,1622433600002,1622433600,...,0,0,0,0,0,0,DNS,1,7062121,Normal flow
1,1,0,75,0,1622433600,120,924,0,1622433600002,1622433600,...,0,0,0,0,0,0,DNS,1,7062122,Normal flow
2,1,0,",",0,1622433600,120,0,0,1622433600007,1622433600,...,0,0,0,0,40,0,Unknown,1,7062123,Normal flow
3,1,0,166,0,1622433600,120,513,0,1622433600090,1622433600,...,0,0,0,0,0,0,DNS,1,7062127,Normal flow
4,1,0,52,0,1622433600,120,26595,26,1622433600097,1622433600,...,0,0,0,0,0,0,TLS.Google,1,7062128,Normal flow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26725241,1,0,",,,,,,,709,,,,,,,,,,,,,,,,,,56,,,,,,,,,,,,,99,...",0,1622458797,120,114419865,114419,1622458912162,1622458912,...,0,0,0,0,0,0,Unknown,1,33792157,Normal flow
26725242,1,0,"52,,,,,,,1635,,,,,,,,,,52,,,,,,,,,,52,,,,,,,11...",0,1622458798,120,104262046,104262,1622458902352,1622458902,...,0,0,0,0,0,0,TLS,1,33792158,Normal flow
26725243,1,0,"612,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1236,,,,,,,,,...",0,1622458799,120,90045366,90045,1622458889634,1622458889,...,0,0,0,0,0,0,TLS.Microsoft365,1,33792243,Normal flow
26725244,1,0,"2822,64995,5036,6186,116250,7597,8299,62399,27...",0,1622458800,120,119940503,119940,1622458919942,1622458919,...,0,0,0,0,0,0,STUN.SkypeCall,1,33792285,Normal flow


In [5]:
#Selección de características

In [6]:
#Separación de datos

In [7]:
#Implementación

In [8]:
#Métricas de evaluación

In [9]:
#Gráfica de la curva ROC

In [10]:
#Evaluación cruzada